# CS 5228


In [3]:
import pandas as pd
import locale
import numpy as np

locale.setlocale(locale.LC_ALL,'')
pd.set_option('display.max_columns', None)

df = pd.read_csv("dataset/Xtrain.csv",dtype= {'Zip': 'object', 'NewExist':'object',
                                      'FranchiseCode':'object','UrbanRural':'object',
                                      'NAICS':'object',}, parse_dates=['ApprovalDate','DisbursementDate'])
# df = pd.read_csv("Xtest.csv")

df['DisbursementGross']= pd.to_numeric(df['DisbursementGross'].map(lambda x: locale.atof(x.strip('$').replace(',','').replace(' ',''))),downcast='float')
df['BalanceGross']= pd.to_numeric(df['BalanceGross'].map(lambda x: locale.atof(x.strip('$').replace(',','').replace(' ',''))),downcast='float')
df['GrAppv']= pd.to_numeric(df['GrAppv'].map(lambda x: locale.atof(x.strip('$').replace(',','').replace(' ',''))),downcast='float')
df['SBA_Appv']= pd.to_numeric(df['SBA_Appv'].map(lambda x: locale.atof(x.strip('$').replace(',','').replace(' ',''))),downcast='float')
df.loc[(df['RevLineCr']!='Y') & (df['RevLineCr']!='N'), 'RevLineCr'] = np.NaN
df.loc[(df['LowDoc']!='Y') & (df['LowDoc']!='N'), 'LowDoc'] = np.NaN

df.loc[(df['NewExist'] != "1.0") & (df['NewExist'] != "2.0"), 'NewExist'] = np.NaN
df['NewExist'] = df['NewExist'].str.strip('.0')

df.loc[(df['ApprovalDate'].dt.year >=2020), 'ApprovalDate'] = df['ApprovalDate'] - pd.DateOffset(years=100)
df.loc[(df['DisbursementDate'].dt.year >=2020), 'DisbursementDate'] = df['DisbursementDate'] - pd.DateOffset(years=100)



df.head(10)


,Id,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementDate,DisbursementGross,BalanceGross,GrAppv,SBA_Appv
0,0,"NPE Realty, LLC",Pembroke Pines,FL,33027,"TD BANK, NATIONAL ASSOCIATION",DE,334516,2010-04-16,2010,130,25,1,2,0,0,1,N,N,2010-05-01,2000000.0,0.0,2000000.0,1500000.0
1,1,"KEVCO CONSTRUCTION, LLC",SANDY,UT,84094,ZIONS FIRST NATIONAL BANK,UT,236115,2010-01-30,2010,33,6,1,1,3,0,1,N,N,2010-02-01,17000.0,0.0,17000.0,17000.0
2,2,EAST L A SHOE REPAIR,LOS ANGELES,CA,90022,BANK OF AMERICA NATL ASSOC,NC,811430,2002-07-29,2002,60,1,1,1,1,1,1,Y,N,2002-08-31,17000.0,0.0,17000.0,8500.0
3,3,"RUSSLER, DANIEL C, MD",LODI,WI,53555,ASSOCIATED BANK NATL ASSOC,WI,0,1994-06-10,1994,160,1,2,0,0,1,0,N,N,1994-07-31,137000.0,0.0,137000.0,123300.0
4,4,AFFORDABLE FAMILY DENISTRY LLC,LEBANON,MO,65536,COMMERCE BANK,MO,621210,2006-02-06,2006,101,2,1,7,2,1,1,NaN,N,2006-03-31,46665.0,0.0,25000.0,12500.0
5,5,COPPER COIN DISCOUNT LIQUOR,PAGOSA SPRINGS,CO,81147,FIRST SOUTHWEST BANK,CO,445310,2008-01-17,2008,272,3,1,1,2,1,2,NaN,N,2008-05-31,400000.0,0.0,500000.0,375000.0
6,6,NICHOLAS MIRABELLA AND PAUL ST,LAS VEGAS,NV,89116,WELLS FARGO BANK NATL ASSOC,SD,811310,2002-09-06,2002,300,5,1,5,5,1,1,NaN,N,2002-10-31,275000.0,0.0,275000.0,206250.0
7,7,SA SA LLC,ELLICOTT CITY,MD,21043,HOWARD BANK,MD,423940,2010-01-15,2010,106,1,2,0,1,0,2,N,N,2010-02-01,75000.0,0.0,75000.0,67500.0
8,8,MICHAEL D CAIN & JANET H CAIN,CONCORD,CA,94518,BANK OF AMERICA NATL ASSOC,NY,621210,1999-12-02,2000,300,5,1,0,5,1,1,NaN,N,1999-12-31,699300.0,0.0,699300.0,524475.0
9,9,Hoke Trucking Ltd.,Acranum,OH,45304,CALIFORNIA BANK & TRUST,CA,484121,2007-01-19,2007,10,15,1,5,15,0,2,N,N,2007-01-31,100000.0,0.0,100000.0,50000.0


In [4]:
df.describe(include=['object', 'bool','datetime64','float64','int64'])

,Id,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementDate
count,50000.000000,49999,50000,50000,50000,49924,49923,50000,50000,50000,50000.000000,50000.000000,49964,50000.00000,50000.000000,50000,50000,35007,49648,49881
unique,NaN,48756,9488,51,13794,2783,52,1118,6934,47,NaN,NaN,2,NaN,NaN,849,3,2,2,2433
top,NaN,SUBWAY,LOS ANGELES,CA,90015,BANK OF AMERICA NATL ASSOC,CA,0,2004-01-30 00:00:00,2007,NaN,NaN,1,NaN,NaN,1,1,N,N,2007-04-30 00:00:00
freq,NaN,55,814,7400,84,5895,7257,8844,53,6228,NaN,NaN,35636,NaN,NaN,31441,30059,21718,44714,591
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1969-04-28 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1969-05-22 00:00:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-04-14 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-04-14 00:00:00
mean,24999.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.856540,9.475280,NaN,9.98176,12.441680,NaN,NaN,NaN,NaN,NaN
std,14433.901067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.066175,59.702302,NaN,266.95799,267.466654,NaN,NaN,NaN,NaN,NaN
min,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0.00000,0.000000,NaN,NaN,NaN,NaN,NaN
25%,12499.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.000000,2.000000,NaN,0.00000,0.000000,NaN,NaN,NaN,NaN,NaN


In [ ]:
df['NewExist'].unique()

In [ ]:
print(df.shape)
print(df.dtypes)

In [ ]:
df.info()